In [9]:
# -*- coding: utf-8 -*-
"""
Implementasi Low-Dimensional Computing (LDC) End-to-End dengan UCI-HAR

Skrip ini membangun dan melatih model LDC menggunakan dataset UCI Human
Activity Recognition, sesuai dengan paper "A Brain-Inspired Low-Dimensional
Computing Classifier for Inference on Tiny Devices".

Perubahan utama dari versi Fashion-MNIST adalah pada pipeline data:
1.  Mengunduh dan mengekstrak dataset UCI-HAR.
2.  Memuat data dari file teks.
3.  Menskalakan fitur ke rentang [0, 255] dan mengkuantisasinya,
    sesuai metodologi paper.
4.  Menyesuaikan hyperparameter model untuk dataset ini.

Setelah training, skrip ini akan mengekspor vektor-vektor yang telah
dipelajari (V, F, dan C) dan memverifikasi akurasinya menggunakan inferensi manual.
"""

# =============================================================================
# 1. Setup dan Import Library
# =============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import os
import requests
import zipfile
from torch.optim.lr_scheduler import StepLR

# =============================================================================
# 2. Definisi Komponen Model LDC (Dengan Perbaikan)
# =============================================================================

# Fungsi aktivasi kustom untuk memastikan konsistensi (sign(0) = 1)
class SignWithZeroToOne(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return torch.where(x == 0, 1.0, torch.sign(x))

    @staticmethod
    def backward(ctx, grad_output):
        # Ini adalah bagian dari Straight-Through Estimator (STE)
        # Melewatkan gradien seolah-olah fungsi ini adalah identitas
        x, = ctx.saved_tensors
        grad_input = grad_output.clone()
        # Gradien hanya mengalir pada nilai antara -1 dan 1
        grad_input[x.abs() > 1.0] = 0
        return grad_input

custom_sign = SignWithZeroToOne.apply

class BinaryLinear(nn.Module):
    """
    Lapisan linear biner menggunakan Straight-Through Estimator (STE)
    untuk memungkinkan training dengan backpropagation.
    """
    def __init__(self, in_features, out_features):
        super(BinaryLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.randn(out_features, in_features) * 0.01)

    def forward(self, x):
        real_weights = self.weight
        # Binarisasi menggunakan STE yang lebih eksplisit
        binary_weights = custom_sign(real_weights)
        y = F.linear(x, binary_weights)
        return y

class ValueBox(nn.Module):
    """
    Jaringan Saraf kecil untuk memetakan nilai skalar (fitur) ke vektor.
    Arsitektur: 1 -> 20 -> Dv
    """
    def __init__(self, output_dim):
        super(ValueBox, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 20),
            nn.Tanh(),
            nn.Linear(20, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)

class LDC_Model(nn.Module):
    """
    Model LDC End-to-End yang menggabungkan semua komponen.
    **Diperbarui dengan Batch Normalization untuk stabilitas.**
    """
    def __init__(self, num_features, dv, df, num_classes, dropout_rate=0.5):
        super(LDC_Model, self).__init__()
        self.num_features = num_features
        self.dv = dv
        self.df = df

        self.value_box = ValueBox(output_dim=dv)

        feature_layer_input_dim = num_features * dv
        # PERBAIKAN: Menambahkan BatchNorm setelah ValueBox
        self.bn_value = nn.BatchNorm1d(feature_layer_input_dim)

        self.feature_layer = BinaryLinear(feature_layer_input_dim, df)

        # PERBAIKAN: Menambahkan BatchNorm setelah FeatureLayer
        self.bn_feature = nn.BatchNorm1d(df)

        self.dropout = nn.Dropout(p=dropout_rate)
        self.class_layer = BinaryLinear(df, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        # Input 'x' sudah dikuantisasi ke 0-255.
        # Normalisasi ke [-1, 1] untuk input ValueBox
        x_normalized = (x / 127.5) - 1.0
        x_reshaped = x_normalized.unsqueeze(-1)
        value_vectors = self.value_box(x_reshaped)

        combined_value_vectors = value_vectors.view(batch_size, -1)
        combined_value_vectors = self.bn_value(combined_value_vectors)

        sample_vector = self.feature_layer(combined_value_vectors)
        sample_vector = self.bn_feature(sample_vector)

        sample_vector = self.dropout(sample_vector)
        sample_vector_bin = custom_sign(sample_vector) # Gunakan fungsi sign kustom

        output_logits = self.class_layer(sample_vector_bin)
        return output_logits

# =============================================================================
# 3. Helper Function untuk Data
# =============================================================================

def download_and_extract_zip(url, dest_path):
    """Mengunduh dan mengekstrak file ZIP dataset."""
    zip_filename = os.path.join(dest_path, 'UCI_HAR_Dataset.zip')
    extract_folder = os.path.join(dest_path, 'UCI HAR Dataset')

    if os.path.exists(extract_folder):
        print(f"Folder dataset sudah ada di '{extract_folder}'")
        return extract_folder

    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    print(f"Mengunduh dataset dari {url}...")
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(zip_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengunduh. Silakan unduh manual dari {url} dan ekstrak ke '{extract_folder}'. Error: {e}")
        return None

    print("Ekstraksi file ZIP...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(dest_path)

    os.remove(zip_filename)
    print("Dataset siap.")
    return extract_folder

# =============================================================================
# 4. Class untuk Inferensi Manual (untuk Verifikasi)
# =============================================================================

class LDCInference:
    """Class untuk melakukan inferensi LDC manual menggunakan NumPy."""
    def __init__(self, vector_dir, training_data_for_scaler):
        # PERBAIKAN: Memuat vektor V REAL-VALUED
        self.V_real = np.load(os.path.join(vector_dir, 'V_real_value_vectors.npy'))
        self.F = np.load(os.path.join(vector_dir, 'F_feature_vectors.npy'))
        self.C = np.load(os.path.join(vector_dir, 'C_class_vectors.npy'))

        # Memuat parameter BatchNorm dan memindahkannya ke CPU
        bn_value_state = torch.load(os.path.join(vector_dir, 'bn_value_state_dict.pth'), map_location='cpu')
        self.bn_value_mean = bn_value_state['running_mean'].numpy()
        self.bn_value_var = bn_value_state['running_var'].numpy()
        self.bn_value_weight = bn_value_state['weight'].numpy()
        self.bn_value_bias = bn_value_state['bias'].numpy()

        bn_feature_state = torch.load(os.path.join(vector_dir, 'bn_feature_state_dict.pth'), map_location='cpu')
        self.bn_feature_mean = bn_feature_state['running_mean'].numpy()
        self.bn_feature_var = bn_feature_state['running_var'].numpy()
        self.bn_feature_weight = bn_feature_state['weight'].numpy()
        self.bn_feature_bias = bn_feature_state['bias'].numpy()

        self.epsilon = 1e-5 # Epsilon default BatchNorm

        # Menggunakan metode normalisasi manual yang sama seperti di training
        self.min_val = np.min(training_data_for_scaler, axis=0)
        self.max_val = np.max(training_data_for_scaler, axis=0)
        self.range_val = self.max_val - self.min_val
        self.range_val[self.range_val == 0] = 1 # Hindari pembagian dengan nol
        self.num_value = 256

    def predict_single(self, raw_sample_vector):
        # Normalisasi dan Kuantisasi
        normalized_sample = (raw_sample_vector - self.min_val) / self.range_val
        quantized_sample = np.clip((normalized_sample * (self.num_value - 1)), 0, self.num_value - 1).astype(int)

        # Langkah 1: ValueBox (menggunakan Vektor REAL)
        value_vectors_combined = self.V_real[quantized_sample].flatten()

        # Langkah 2: Terapkan BatchNorm untuk Value
        bn_value_out = (value_vectors_combined - self.bn_value_mean) / np.sqrt(self.bn_value_var + self.epsilon)
        bn_value_out = bn_value_out * self.bn_value_weight + self.bn_value_bias

        # Langkah 3: FeatureLayer
        sample_vector_S_real = bn_value_out @ self.F.T

        # Langkah 4: Terapkan BatchNorm untuk Feature
        bn_feature_out = (sample_vector_S_real - self.bn_feature_mean) / np.sqrt(self.bn_feature_var + self.epsilon)
        bn_feature_out = bn_feature_out * self.bn_feature_weight + self.bn_feature_bias

        # Langkah 5: Binarisasi
        sample_vector_S_bin = np.sign(bn_feature_out)
        sample_vector_S_bin[sample_vector_S_bin == 0] = 1

        # Langkah 6: ClassLayer
        similarity_scores = sample_vector_S_bin @ self.C.T
        return np.argmax(similarity_scores)

# =============================================================================
# 5. Fungsi Training dan Eksekusi Utama
# =============================================================================

def main():
    """
    Fungsi utama untuk menjalankan seluruh pipeline:
    data loading, inisialisasi model, training, ekspor, dan verifikasi.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Menggunakan device: {device}')

    # --- Persiapan Data (UCI-HAR) ---
    zip_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip'
    data_dir = download_and_extract_zip(zip_url, './data')
    if data_dir is None: return

    X_train_raw = np.loadtxt(os.path.join(data_dir, 'train', 'X_train.txt'), dtype=np.float32)
    y_train_raw = np.loadtxt(os.path.join(data_dir, 'train', 'y_train.txt'), dtype=np.int64)
    X_test_raw = np.loadtxt(os.path.join(data_dir, 'test', 'X_test.txt'), dtype=np.float32)
    y_test_raw = np.loadtxt(os.path.join(data_dir, 'test', 'y_test.txt'), dtype=np.int64)

    # Normalisasi dan Kuantisasi Manual
    num_value = 256
    min_val = np.min(X_train_raw, axis=0)
    max_val = np.max(X_train_raw, axis=0)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1

    X_train_normalized = (X_train_raw - min_val) / range_val
    X_test_normalized = (X_test_raw - min_val) / range_val

    X_train_quantized = np.clip((X_train_normalized * (num_value - 1)), 0, num_value - 1).astype(int)
    X_test_quantized = np.clip((X_test_normalized * (num_value - 1)), 0, num_value - 1).astype(int)

    X_train = X_train_quantized.astype(np.float32)
    X_test = X_test_quantized.astype(np.float32)

    y_train = y_train_raw - 1
    y_test = y_test_raw - 1

    # --- Hyperparameter Sesuai Paper LDC ---
    BATCH_SIZE = 64
    EPOCHS = 100
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.0001
    DROPOUT_RATE = 0.5

    train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

    test_set = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

    print(f'Jumlah data training: {len(train_set)}')
    print(f'Jumlah data testing: {len(test_set)}\n')

    # --- Training Model LDC ---
    NUM_FEATURES = 561
    NUM_CLASSES = 6
    DV = 8
    DF = 128 # Sesuai paper

    model = LDC_Model(NUM_FEATURES, DV, DF, NUM_CLASSES, dropout_rate=DROPOUT_RATE).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

    print("Model LDC disesuaikan dengan paper untuk UCI-HAR:")
    print(f"Features: {NUM_FEATURES}, Classes: {NUM_CLASSES}, Dv: {DV}, Df: {DF}")
    print("\nMemulai proses training...")

    best_accuracy = 0.0
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')

        for data, labels in progress_bar:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix({'loss': f'{running_loss / (progress_bar.n + 1):.4f}'})

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, labels in test_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        current_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}, Akurasi Test (PyTorch): {current_accuracy:.2f}%')

        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            # Simpan model terbaik untuk diekspor nanti
            torch.save(model.state_dict(), 'best_model.pth')

        scheduler.step()

    print(f"\nAkurasi terbaik yang dicapai: {best_accuracy:.2f}%")

    # =============================================================================
    # 6. Ekspor Vektor Model (dari model terbaik)
    # =============================================================================
    print("\nMemuat model terbaik untuk ekspor...")
    # Buat instance model baru dan muat state_dict terbaik
    best_model = LDC_Model(NUM_FEATURES, DV, DF, NUM_CLASSES, dropout_rate=DROPOUT_RATE).to(device)
    best_model.load_state_dict(torch.load('best_model.pth'))
    best_model.eval()

    print("Mengekspor vektor model...")

    output_dir = "exported_vectors_ucihar"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with torch.no_grad():
        possible_values = torch.arange(0, 256, device=device).float().unsqueeze(1)
        normalized_values = (possible_values / 127.5) - 1.0
        # PERBAIKAN: Ekspor Vektor V REAL-VALUED
        value_vectors_V_real = best_model.value_box(normalized_values)
        np.save(os.path.join(output_dir, 'V_real_value_vectors.npy'), value_vectors_V_real.cpu().numpy())

    with torch.no_grad():
        feature_vectors_F = custom_sign(best_model.feature_layer.weight)
        np.save(os.path.join(output_dir, 'F_feature_vectors.npy'), feature_vectors_F.cpu().numpy())

    with torch.no_grad():
        class_vectors_C = custom_sign(best_model.class_layer.weight)
        np.save(os.path.join(output_dir, 'C_class_vectors.npy'), class_vectors_C.cpu().numpy())

    # Ekspor parameter BatchNorm
    torch.save(best_model.bn_value.state_dict(), os.path.join(output_dir, 'bn_value_state_dict.pth'))
    torch.save(best_model.bn_feature.state_dict(), os.path.join(output_dir, 'bn_feature_state_dict.pth'))

    print(f"Vektor dan parameter BatchNorm berhasil diekspor ke folder '{output_dir}'.")

    # =============================================================================
    # 7. Verifikasi Menggunakan Inferensi Manual
    # =============================================================================
    print("\nMemulai verifikasi dengan inferensi manual pada seluruh data tes...")
    engine = LDCInference(output_dir, X_train_raw)

    manual_correct = 0
    for i in tqdm(range(len(X_test_raw)), desc="Verifikasi Manual"):
        prediction = engine.predict_single(X_test_raw[i])
        if prediction == y_test[i]:
            manual_correct += 1

    manual_accuracy = 100 * manual_correct / len(X_test_raw)

    print("\n--- Hasil Verifikasi ---")
    print(f"Akurasi model PyTorch (terbaik): {best_accuracy:.2f}%")
    print(f"Akurasi inferensi manual (NumPy):      {manual_accuracy:.2f}%")

    # Toleransi bisa sedikit lebih besar karena perbedaan minor antara STE dan NumPy
    if abs(best_accuracy - manual_accuracy) < 0.5:
        print("✅ Verifikasi BERHASIL: Akurasi cocok.")
    else:
        print("❌ Verifikasi GAGAL: Terdapat perbedaan signifikan pada akurasi.")

    # =============================================================================
    # 8. (FIXED-POINT) Ekspor Parameter & Data Uji untuk Implementasi C
    # =============================================================================


    print("\nMemulai ekspor untuk implementasi Fixed-Point...")

    # --- Konfigurasi ---
    SCALE_FACTOR = 4096
    # Ekspor sampel pertama (indeks 0) dari data tes
    TEST_SAMPLE_IDX = 0

    # --- Perhitungan Parameter Fixed-Point ---
    # Variabel X_train_raw, X_test_raw, dan y_test sudah ada dari proses sebelumnya
    min_val = np.min(X_train_raw, axis=0)
    max_val = np.max(X_train_raw, axis=0)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1e-9

    min_val_scaled = (min_val * SCALE_FACTOR).astype(np.int32)
    inv_range_val_scaled = ((1.0 / range_val) * SCALE_FACTOR).astype(np.int32)

    # Ambil data mentah (float) dan label yang sesuai
    test_data_sample_raw = X_test_raw[TEST_SAMPLE_IDX]
    # REPLACE IT WITH THIS:
    # Use the zero-indexed y_test, which is consistent with the model's training
    y_test = y_test_raw - 1
    test_data_actual_label = y_test[TEST_SAMPLE_IDX]

    print(f"Data uji mentah (float) yang diekspor adalah untuk kelas: {test_data_actual_label}")

    # --- Proses Ekspor ke File C ---

    # Ekspor Parameter Model (model_params_scaled.h)
    header_path = "model_params_scaled.h"
    with open(header_path, "w") as f:
        f.write("#ifndef MODEL_PARAMS_SCALED_H_\n#define MODEL_PARAMS_SCALED_H_\n\n#include <stdint.h>\n\n")
        f.write(f"#define FIXED_POINT_SCALE_FACTOR {SCALE_FACTOR}\n\n")
        f.write(f"const int32_t min_val_scaled[{len(min_val_scaled)}] = {{ ")
        f.write(", ".join(map(str, min_val_scaled)))
        f.write(" };\n\n")
        f.write(f"const int32_t inv_range_val_scaled[{len(inv_range_val_scaled)}] = {{ ")
        f.write(", ".join(map(str, inv_range_val_scaled)))
        f.write(" };\n\n#endif // MODEL_PARAMS_SCALED_H_\n")
    print(f"Berhasil mengekspor '{header_path}'")

    # Ekspor Data Uji (test_data.h dan test_data.c)
    header_path = "test_data.h"
    with open(header_path, "w") as f:
        f.write("#ifndef TEST_DATA_H_\n#define TEST_DATA_H_\n\n#include <stdint.h>\n\n")
        f.write(f"#define TEST_DATA_SAMPLE_LENGTH {len(test_data_sample_raw)}\n")
        f.write("extern const float test_data_sample[];\n")
        f.write(f"extern const uint8_t test_data_actual_label;\n\n#endif // TEST_DATA_H_\n")
    print(f"Berhasil mengekspor '{header_path}'")

    source_path = "test_data.c"
    with open(source_path, "w") as f:
        f.write('#include "test_data.h"\n\n')
        f.write(f"const uint8_t test_data_actual_label = {test_data_actual_label};\n\n")
        f.write(f"const float test_data_sample[TEST_DATA_SAMPLE_LENGTH] = {{\n    ")
        f.write(", ".join([f"{val:.8f}f" for val in test_data_sample_raw]))
        f.write("\n};\n")
    print(f"Berhasil mengekspor '{source_path}'")

if __name__ == '__main__':
    main()



Menggunakan device: cuda
Folder dataset sudah ada di './data/UCI HAR Dataset'
Jumlah data training: 7352
Jumlah data testing: 2947

Model LDC disesuaikan dengan paper untuk UCI-HAR:
Features: 561, Classes: 6, Dv: 8, Df: 128

Memulai proses training...


Epoch 1/100: 100%|██████████| 115/115 [00:00<00:00, 217.12it/s, loss=3.1214]


Epoch [1/100], Loss: 3.0671, Akurasi Test (PyTorch): 82.63%


Epoch 2/100: 100%|██████████| 115/115 [00:00<00:00, 214.11it/s, loss=1.7490]


Epoch [2/100], Loss: 1.7033, Akurasi Test (PyTorch): 89.04%


...

Epoch [99/100], Loss: 0.0443, Akurasi Test (PyTorch): 96.03%


Epoch 100/100: 100%|██████████| 115/115 [00:00<00:00, 179.59it/s, loss=0.0483]


Epoch [100/100], Loss: 0.0466, Akurasi Test (PyTorch): 95.76%

Akurasi terbaik yang dicapai: 96.27%

Memuat model terbaik untuk ekspor...
Mengekspor vektor model...
Vektor dan parameter BatchNorm berhasil diekspor ke folder 'exported_vectors_ucihar'.

Memulai verifikasi dengan inferensi manual pada seluruh data tes...


Verifikasi Manual: 100%|██████████| 2947/2947 [00:00<00:00, 5220.82it/s]


--- Hasil Verifikasi ---
Akurasi model PyTorch (terbaik): 96.27%
Akurasi inferensi manual (NumPy):      96.27%
✅ Verifikasi BERHASIL: Akurasi cocok.

Memulai ekspor untuk implementasi Fixed-Point...
Data uji mentah (float) yang diekspor adalah untuk kelas: 4
Berhasil mengekspor 'model_params_scaled.h'
Berhasil mengekspor 'test_data.h'
Berhasil mengekspor 'test_data.c'


In [10]:
# -*- coding: utf-8 -*-
"""
Skrip untuk Melakukan Inferensi Langsung pada Satu Data LDC

Skrip ini menyediakan class 'LDCInference' yang membungkus semua
logika dan aset model (vektor V, F, C, dan parameter BatchNorm) untuk
memudahkan prediksi pada data mentah tunggal.

Cara kerja:
1.  Inisialisasi class `LDCInference`. Ini akan memuat semua yang dibutuhkan.
2.  Siapkan satu sampel data mentah (misalnya, satu baris dari file teks UCI-HAR).
3.  Panggil metode `predict` pada sampel tersebut untuk mendapatkan hasilnya.
"""
import numpy as np
import os
import torch # Diperlukan hanya untuk memuat state_dict BatchNorm
import requests
import zipfile

# =============================================================================
# 1. Class Inference Engine (Final & Terverifikasi)
# =============================================================================

class LDCInference:
    """
    Sebuah class untuk melakukan inferensi LDC pada data baru.

    Class ini menangani pemuatan model (vektor V, F, C) dan prapemrosesan data
    (scaling dan kuantisasi) yang konsisten dengan saat training.
    """
    def __init__(self, vector_dir, training_data_for_scaler):
        """
        Inisialisasi inference engine.

        Args:
            vector_dir (str): Path ke folder berisi file V, F, C .npy dan state_dict.
            training_data_for_scaler (np.array): Data training mentah (X_train_raw)
                yang diperlukan untuk me-fit scaler agar prapemrosesan konsisten.
        """
        print("Menginisialisasi LDC Inference Engine...")
        if not os.path.exists(vector_dir):
            raise FileNotFoundError(f"Folder vektor '{vector_dir}' tidak ditemukan.")

        # 1. Memuat vektor V (real-valued), F (biner), C (biner)
        self.V_real = np.load(os.path.join(vector_dir, 'V_real_value_vectors.npy'))
        self.F = np.load(os.path.join(vector_dir, 'F_feature_vectors.npy'))
        self.C = np.load(os.path.join(vector_dir, 'C_class_vectors.npy'))

        # 2. Memuat parameter BatchNorm
        bn_value_state = torch.load(os.path.join(vector_dir, 'bn_value_state_dict.pth'), map_location='cpu')
        self.bn_value_mean = bn_value_state['running_mean'].numpy()
        self.bn_value_var = bn_value_state['running_var'].numpy()
        self.bn_value_weight = bn_value_state['weight'].numpy()
        self.bn_value_bias = bn_value_state['bias'].numpy()

        bn_feature_state = torch.load(os.path.join(vector_dir, 'bn_feature_state_dict.pth'), map_location='cpu')
        self.bn_feature_mean = bn_feature_state['running_mean'].numpy()
        self.bn_feature_var = bn_feature_state['running_var'].numpy()
        self.bn_feature_weight = bn_feature_state['weight'].numpy()
        self.bn_feature_bias = bn_feature_state['bias'].numpy()

        self.epsilon = 1e-5

        # 3. Menyiapkan scaler data
        self.min_val = np.min(training_data_for_scaler, axis=0)
        self.max_val = np.max(training_data_for_scaler, axis=0)
        self.range_val = self.max_val - self.min_val
        self.range_val[self.range_val == 0] = 1
        self.num_value = 256

        # 4. Nama kelas untuk interpretasi
        self.class_names = [
            'WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS',
            'SITTING', 'STANDING', 'LAYING'
        ]
        print("Engine siap.")

    def predict(self, raw_sample_vector):
        """
        Melakukan inferensi lengkap pada satu sampel data mentah.

        Args:
            raw_sample_vector (np.array): Vektor fitur 1D dari data mentah.

        Returns:
            str: Nama kelas yang diprediksi.
            np.array: Skor kemiripan untuk semua kelas.
        """
        # Prapemrosesan: Normalisasi dan Kuantisasi
        normalized_sample = (raw_sample_vector - self.min_val) / self.range_val
        quantized_sample = np.clip((normalized_sample * (self.num_value - 1)), 0, self.num_value - 1).astype(int)

        # Langkah 1: ValueBox (menggunakan Vektor REAL)
        value_vectors_combined = self.V_real[quantized_sample].flatten()

        # Langkah 2: Terapkan BatchNorm untuk Value
        bn_value_out = (value_vectors_combined - self.bn_value_mean) / np.sqrt(self.bn_value_var + self.epsilon)
        bn_value_out = bn_value_out * self.bn_value_weight + self.bn_value_bias

        # Langkah 3: FeatureLayer
        sample_vector_S_real = bn_value_out @ self.F.T

        # Langkah 4: Terapkan BatchNorm untuk Feature
        bn_feature_out = (sample_vector_S_real - self.bn_feature_mean) / np.sqrt(self.bn_feature_var + self.epsilon)
        bn_feature_out = bn_feature_out * self.bn_feature_weight + self.bn_feature_bias

        # Langkah 5: Binarisasi
        sample_vector_S_bin = np.sign(bn_feature_out)
        sample_vector_S_bin[sample_vector_S_bin == 0] = 1

        # Langkah 6: ClassLayer
        similarity_scores = sample_vector_S_bin @ self.C.T
        predicted_idx = np.argmax(similarity_scores)

        return self.class_names[predicted_idx], similarity_scores

# =============================================================================
# 2. Contoh Penggunaan
# =============================================================================

def get_data_for_setup(url='https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip', dest_path='./data'):
    """Fungsi helper untuk mendapatkan data yang dibutuhkan untuk inisialisasi."""
    extract_folder = os.path.join(dest_path, 'UCI HAR Dataset')
    if not os.path.exists(extract_folder):
        # Logika download sederhana jika data tidak ada
        print("Data UCI-HAR tidak ditemukan. Silakan jalankan 'ldc_ucihar.py' terlebih dahulu.")
        return None, None, None

    X_train_raw = np.loadtxt(os.path.join(extract_folder, 'train', 'X_train.txt'), dtype=np.float32)
    X_test_raw = np.loadtxt(os.path.join(extract_folder, 'test', 'X_test.txt'), dtype=np.float32)
    y_test_raw = np.loadtxt(os.path.join(extract_folder, 'test', 'y_test.txt'), dtype=np.int64)
    return X_train_raw, X_test_raw, y_test_raw

def main():
    print("--- Contoh Penggunaan LDC Inference Engine ---")

    # --- Persiapan ---
    vector_dir = "exported_vectors_ucihar"
    X_train_raw, X_test_raw, y_test_raw = get_data_for_setup()

    if X_train_raw is None: return

    # --- Inisialisasi Engine ---
    try:
        engine = LDCInference(vector_dir, X_train_raw)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Pastikan Anda telah menjalankan 'ldc_ucihar.py' untuk menghasilkan vektor model.")
        return

    print("\n--- Melakukan Inferensi pada Data Baru ---")
    # Di aplikasi nyata, 'sample_to_test' akan berasal dari sensor atau input lain.
    # Kita ambil satu sampel acak dari data tes sebagai contoh.
    sample_index = np.random.randint(0, len(X_test_raw))
    sample_to_test = X_test_raw[sample_index]
    actual_label_idx = y_test_raw[sample_index] - 1
    actual_label_name = engine.class_names[actual_label_idx]

    print(f"Menguji sampel data #{sample_index}")
    print(f"Label aktual: {actual_label_name}")

    # --- Lakukan Prediksi ---
    predicted_class, similarity_scores = engine.predict(sample_to_test)

    print("\n--- Hasil Inferensi ---")
    print(f"Prediksi Model: '{predicted_class}'")

    if predicted_class == actual_label_name:
        print("✅ Prediksi BENAR!")
    else:
        print(f"❌ Prediksi SALAH.")

if __name__ == '__main__':
    main()


--- Contoh Penggunaan LDC Inference Engine ---
Menginisialisasi LDC Inference Engine...
Engine siap.

--- Melakukan Inferensi pada Data Baru ---
Menguji sampel data #1140
Label aktual: WALKING

--- Hasil Inferensi ---
Prediksi Model: 'WALKING'
✅ Prediksi BENAR!


# NEW REVISE

In [11]:
# -*- coding: utf-8 -*-
"""
Konverter Model Universal MicroVSA (Diperbaiki)

Skrip ini mengambil vektor model (C, F, dan V) yang telah dilatih dan
disimpan dalam format NumPy (.npy), lalu mengubahnya menjadi sepasang file
.h dan .c yang siap digunakan oleh library MicroVSA di microcontroller.

Versi ini memperbaiki logika preprocessor di file .c yang dihasilkan untuk
menghindari error 'conflicting types' dan '#else without #if'.

File yang Dihasilkan:
1.  **model.h**: File header yang berisi deklarasi 'extern' untuk variabel
    model dan makro untuk dimensi.
2.  **model.c**: File sumber yang berisi definisi aktual (data heksadesimal)
    dari semua matriks model.

CARA MENGGUNAKAN:
1.  Letakkan file .npy Anda (C_class_vectors.npy, dll.) di folder yang sama.
2.  Jalankan skrip: `python universal_converter.py`
3.  File `model.h` dan `model.c` akan dibuat/ditimpa dengan versi yang benar.
"""

import numpy as np
import os
import textwrap

# --- PENGATURAN ---
VECTOR_DIR = "/content/exported_vectors_ucihar"
OUTPUT_HEADER_FILENAME = "model.h"
OUTPUT_SOURCE_FILENAME = "model.c"

VAR_NAME_C = "MICROVSA_MODEL_C"
VAR_NAME_F = "MICROVSA_MODEL_F"
VAR_NAME_V = "MICROVSA_MODEL_V"

C_QUALIFIER = "MODEL_C_QUALIFIER"
F_QUALIFIER = "MODEL_F_QUALIFIER"
V_QUALIFIER = "MODEL_V_QUALIFIER"

# --- Bentuk Matriks Input yang Diharapkan ---
# - C_class_vectors.npy:      (JUMLAH_KELAS, DF)
# - F_feature_vectors.npy:      (JUMLAH_FITUR, DF)
# - V_real_value_vectors.npy: (256, DV)

def pack_bipolar_array(bipolar_arr, word_size_bits):
    if bipolar_arr.ndim != 2:
        raise ValueError("Input array harus 2D.")

    rows, cols = bipolar_arr.shape

    if cols % word_size_bits != 0:
        padding_needed = word_size_bits - (cols % word_size_bits)
        padding_values = np.ones((rows, padding_needed), dtype=bipolar_arr.dtype)
        padded_arr = np.hstack((bipolar_arr, padding_values))
        print(f"  [INFO] Menambahkan padding pada matriks shape {bipolar_arr.shape} -> {padded_arr.shape} untuk word_size {word_size_bits}")
        bipolar_arr = padded_arr
        cols = bipolar_arr.shape[1]

    # Konversi bipolar ke biner: -1 -> 1, 1 -> 0 (untuk ekuivalensi XOR)
    binary_arr = np.where(bipolar_arr <= 0, 1, 0).astype(np.uint8)
    packed_8bit = np.packbits(binary_arr, axis=-1)

    if word_size_bits == 8:
        return packed_8bit

    dtype = np.uint16 if word_size_bits == 16 else np.uint32
    bytes_per_word = word_size_bits // 8

    if packed_8bit.shape[1] % bytes_per_word != 0:
        raise ValueError("Packing error, shape tidak cocok setelah padding.")

    # Ubah urutan byte ke big-endian sebelum melihatnya sebagai tipe data yang lebih besar
    packed_8bit_big_endian = packed_8bit.reshape(rows, -1, bytes_per_word)

    # Buat array kosong dengan tipe data target
    result = np.zeros((packed_8bit_big_endian.shape[0], packed_8bit_big_endian.shape[1]), dtype=dtype)

    # Salin data byte-per-byte
    for i in range(bytes_per_word):
        result |= packed_8bit_big_endian[:, :, i].astype(dtype) << ((bytes_per_word - 1 - i) * 8)

    return result

def format_array_to_c_string(packed_arr, var_name, qualifier, c_type):
    hex_formatter = {
        'uint8_t': lambda x: f"0x{x:02x}",
        'uint16_t': lambda x: f"0x{x:04x}",
        'uint32_t': lambda x: f"0x{x:08x}",
    }[c_type]
    flat_list = [hex_formatter(val) for val in packed_arr.flatten()]
    body = ", ".join(flat_list)
    wrapped_body = "\n" + textwrap.fill(body, width=80, initial_indent="    ", subsequent_indent="    ") + "\n"
    return f"{qualifier} {c_type} {var_name}[] = {{{wrapped_body}}};"

def generate_c_code_for_matrix(f_source, matrix, var_name, qualifier, word_size):
    c_type = f"uint{word_size}_t"
    packed_normal = pack_bipolar_array(matrix, word_size)
    c_string_normal = format_array_to_c_string(packed_normal, var_name, qualifier, c_type)

    transposed_matrix = matrix.T
    packed_transposed = pack_bipolar_array(transposed_matrix, word_size)
    c_string_transposed = format_array_to_c_string(packed_transposed, var_name, qualifier, c_type)

    var_suffix = var_name.split('_')[-1]
    f_source.write(f"#ifndef MODEL_TRANSPOSE_{var_suffix}\n")
    f_source.write(c_string_normal)
    f_source.write("\n#else\n")
    f_source.write(c_string_transposed)
    f_source.write("\n#endif\n\n")

def main():
    print("Memulai proses konversi model MicroVSA...")
    try:
        c_vec = np.load(os.path.join(VECTOR_DIR, 'C_class_vectors.npy'))
        f_vec = np.load(os.path.join(VECTOR_DIR, 'F_feature_vectors.npy'))
        v_vec_real = np.load(os.path.join(VECTOR_DIR, 'V_real_value_vectors.npy'))
        print("Berhasil memuat file .npy.")

        v_vec = np.sign(v_vec_real); v_vec[v_vec == 0] = -1
        num_classes, df = c_vec.shape
        _, dv = v_vec.shape

        if f_vec.shape[1] == df:
            num_features = f_vec.shape[0]
        elif f_vec.shape[0] == df:
            print("  [INFO] Matriks F terdeteksi dalam format (DF, num_features). Mentransposisinya ke (num_features, DF).")
            f_vec = f_vec.T
            num_features = f_vec.shape[0]
        else:
            raise ValueError(f"Dimensi DF dari matriks F ({f_vec.shape}) tidak cocok dengan matriks C ({c_vec.shape}).")

        print(f"  - Dimensi terdeteksi: DF={df}, DV={dv}, Classes={num_classes}, Features={num_features}")
    except Exception as e:
        print(f"\nERROR: Gagal memproses file: {e}")
        return

    # --- Tulis File Header (.h) ---
    with open(OUTPUT_HEADER_FILENAME, 'w') as f_header:
        header_guard = f"MODEL_{os.path.basename(OUTPUT_HEADER_FILENAME).upper().replace('.', '_')}_"
        f_header.write(f"#ifndef {header_guard}\n#define {header_guard}\n\n")
        f_header.write("#include <stdint.h>\n")
        f_header.write("#include \"microvsa_config.h\"\n\n")
        f_header.write(f"#define MICROVSA_MODEL_FHV_DIMENSION_BIT {df}\n")
        f_header.write(f"#if MICROVSA_IMPL_WORDSIZE == 8\n#define MICROVSA_MODEL_FHV_DIMENSION_WORD {df // 8}\n")
        f_header.write(f"#elif MICROVSA_IMPL_WORDSIZE == 16\n#define MICROVSA_MODEL_FHV_DIMENSION_WORD {df // 16}\n")
        f_header.write(f"#elif MICROVSA_IMPL_WORDSIZE == 32\n#define MICROVSA_MODEL_FHV_DIMENSION_WORD {df // 32}\n")
        f_header.write("#else\n# error Unsupported MICROVSA_IMPL_WORDSIZE\n#endif\n\n")
        f_header.write(f"#define MICROVSA_MODEL_NUM_CLASS {num_classes}\n")
        f_header.write(f"#define MICROVSA_MODEL_NUM_FEATURE {num_features}\n\n")
        f_header.write(f"#ifdef MODEL_C_IN_RAM\n#define {C_QUALIFIER}\n#else\n#define {C_QUALIFIER} const\n#endif\n")
        f_header.write(f"#ifdef MODEL_F_IN_RAM\n#define {F_QUALIFIER}\n#else\n#define {F_QUALIFIER} const\n#endif\n")
        f_header.write(f"#ifdef MODEL_V_IN_RAM\n#define {V_QUALIFIER}\n#else\n#define {V_QUALIFIER} const\n#endif\n\n")
        f_header.write("#if MICROVSA_IMPL_WORDSIZE == 8\n")
        f_header.write(f"extern {C_QUALIFIER} uint8_t {VAR_NAME_C}[];\n")
        f_header.write(f"extern {F_QUALIFIER} uint8_t {VAR_NAME_F}[];\n")
        f_header.write(f"extern {V_QUALIFIER} uint8_t {VAR_NAME_V}[];\n")
        f_header.write("#elif MICROVSA_IMPL_WORDSIZE == 16\n")
        f_header.write(f"extern {C_QUALIFIER} uint16_t {VAR_NAME_C}[];\n")
        f_header.write(f"extern {F_QUALIFIER} uint16_t {VAR_NAME_F}[];\n")
        f_header.write(f"extern {V_QUALIFIER} uint16_t {VAR_NAME_V}[];\n")
        f_header.write("#elif MICROVSA_IMPL_WORDSIZE == 32\n")
        f_header.write(f"extern {C_QUALIFIER} uint32_t {VAR_NAME_C}[];\n")
        f_header.write(f"extern {F_QUALIFIER} uint32_t {VAR_NAME_F}[];\n")
        f_header.write(f"extern {V_QUALIFIER} uint32_t {VAR_NAME_V}[];\n")
        f_header.write("#endif\n\n")
        f_header.write(f"#endif // {header_guard}\n")

    # --- Tulis File Sumber (.c) ---
    with open(OUTPUT_SOURCE_FILENAME, 'w') as f_source:
        f_source.write(f"// File sumber model MicroVSA, dibuat secara otomatis.\n")
        f_source.write(f"#include \"{OUTPUT_HEADER_FILENAME}\"\n\n")

        # --- Blok untuk 8-bit ---
        f_source.write("#if MICROVSA_IMPL_WORDSIZE == 8\n")
        print("Memproses untuk arsitektur 8-bit...")
        packed_v_8 = pack_bipolar_array(v_vec, 8)
        f_source.write(format_array_to_c_string(packed_v_8, VAR_NAME_V, V_QUALIFIER, 'uint8_t') + "\n")
        generate_c_code_for_matrix(f_source, f_vec, VAR_NAME_F, F_QUALIFIER, 8)
        generate_c_code_for_matrix(f_source, c_vec, VAR_NAME_C, C_QUALIFIER, 8)

        # --- Blok untuk 16-bit ---
        f_source.write("#elif MICROVSA_IMPL_WORDSIZE == 16\n")
        print("Memproses untuk arsitektur 16-bit...")
        packed_v_16 = pack_bipolar_array(v_vec, 16)
        f_source.write(format_array_to_c_string(packed_v_16, VAR_NAME_V, V_QUALIFIER, 'uint16_t') + "\n")
        generate_c_code_for_matrix(f_source, f_vec, VAR_NAME_F, F_QUALIFIER, 16)
        generate_c_code_for_matrix(f_source, c_vec, VAR_NAME_C, C_QUALIFIER, 16)

        # --- Blok untuk 32-bit ---
        f_source.write("#elif MICROVSA_IMPL_WORDSIZE == 32\n")
        print("Memproses untuk arsitektur 32-bit...")
        packed_v_32 = pack_bipolar_array(v_vec, 32)
        f_source.write(format_array_to_c_string(packed_v_32, VAR_NAME_V, V_QUALIFIER, 'uint32_t') + "\n")
        generate_c_code_for_matrix(f_source, f_vec, VAR_NAME_F, F_QUALIFIER, 32)
        generate_c_code_for_matrix(f_source, c_vec, VAR_NAME_C, C_QUALIFIER, 32)

        f_source.write("#endif\n")

    print(f"\n✅ Berhasil! File '{OUTPUT_HEADER_FILENAME}' dan '{OUTPUT_SOURCE_FILENAME}' telah diperbarui.")


if __name__ == '__main__':
    main()



Memulai proses konversi model MicroVSA...
Berhasil memuat file .npy.
  [INFO] Matriks F terdeteksi dalam format (DF, num_features). Mentransposisinya ke (num_features, DF).
  - Dimensi terdeteksi: DF=128, DV=8, Classes=6, Features=4488
Memproses untuk arsitektur 8-bit...
  [INFO] Menambahkan padding pada matriks shape (128, 6) -> (128, 8) untuk word_size 8
Memproses untuk arsitektur 16-bit...
  [INFO] Menambahkan padding pada matriks shape (256, 8) -> (256, 16) untuk word_size 16
  [INFO] Menambahkan padding pada matriks shape (128, 4488) -> (128, 4496) untuk word_size 16
  [INFO] Menambahkan padding pada matriks shape (128, 6) -> (128, 16) untuk word_size 16
Memproses untuk arsitektur 32-bit...
  [INFO] Menambahkan padding pada matriks shape (256, 8) -> (256, 32) untuk word_size 32
  [INFO] Menambahkan padding pada matriks shape (128, 4488) -> (128, 4512) untuk word_size 32
  [INFO] Menambahkan padding pada matriks shape (128, 6) -> (128, 32) untuk word_size 32

✅ Berhasil! File 'mode

In [12]:
# =============================================================================
# 10. (FIXED-POINT + DEBUG) Ekspor Parameter & Data Uji untuk Implementasi C
# =============================================================================
import numpy as np
from os.path import join

print("Memulai ekspor untuk implementasi Fixed-Point...")

# --- Konfigurasi ---
SCALE_FACTOR = 4096
TEST_SAMPLE_IDX = 0

# --- Perhitungan Parameter Fixed-Point ---
try:
    data_dir = './data/UCI HAR Dataset';
    X_train_raw = np.loadtxt(os.path.join(data_dir, 'train', 'X_train.txt'), dtype=np.float32)
    y_train_raw = np.loadtxt(os.path.join(data_dir, 'train', 'y_train.txt'), dtype=np.int64)
    X_test_raw = np.loadtxt(os.path.join(data_dir, 'test', 'X_test.txt'), dtype=np.float32)
    y_test_raw = np.loadtxt(os.path.join(data_dir, 'test', 'y_test.txt'), dtype=np.int64)
    min_val = np.min(X_train_raw, axis=0)
    max_val = np.max(X_train_raw, axis=0)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1e-9

    min_val_scaled = (min_val * SCALE_FACTOR).astype(np.int32)
    inv_range_val_scaled = ((1.0 / range_val) * SCALE_FACTOR).astype(np.int32)
    test_data_sample_raw = X_test_raw[TEST_SAMPLE_IDX]
    # REPLACE IT WITH THIS:
    # Use the zero-indexed y_test, which is consistent with the model's training
    y_test = y_test_raw - 1
    test_data_actual_label = y_test[TEST_SAMPLE_IDX]
    print(f"Data uji yang diekspor adalah untuk kelas: {test_data_actual_label}")

except NameError as e:
    print(f"ERROR: Pastikan 'X_train_raw' dan 'y_test' sudah ada. Error: {e}")
    raise

# --- Proses Ekspor ke File C (Tidak ada perubahan di sini) ---
# (Kode ekspor file .h dan .c tetap sama seperti sebelumnya)
header_path = "model_params_scaled.h"
with open(header_path, "w") as f:
    f.write("#ifndef MODEL_PARAMS_SCALED_H_\n#define MODEL_PARAMS_SCALED_H_\n\n#include <stdint.h>\n\n")
    f.write(f"#define FIXED_POINT_SCALE_FACTOR {SCALE_FACTOR}\n\n")
    f.write(f"const int32_t min_val_scaled[{len(min_val_scaled)}] = {{ ")
    f.write(", ".join(map(str, min_val_scaled)))
    f.write(" };\n\n")
    f.write(f"const int32_t inv_range_val_scaled[{len(inv_range_val_scaled)}] = {{ ")
    f.write(", ".join(map(str, inv_range_val_scaled)))
    f.write(" };\n\n#endif // MODEL_PARAMS_SCALED_H_\n")
print(f"Berhasil mengekspor '{header_path}'")
header_path = "test_data.h"
with open(header_path, "w") as f:
    f.write("#ifndef TEST_DATA_H_\n#define TEST_DATA_H_\n\n#include <stdint.h>\n\n")
    f.write(f"#define TEST_DATA_SAMPLE_LENGTH {len(test_data_sample_raw)}\n")
    f.write("extern const float test_data_sample[];\n")
    f.write(f"extern const uint8_t test_data_actual_label;\n\n#endif // TEST_DATA_H_\n")
print(f"Berhasil mengekspor '{header_path}'")
source_path = "test_data.c"
with open(source_path, "w") as f:
    f.write('#include "test_data.h"\n\n')
    f.write(f"const uint8_t test_data_actual_label = {test_data_actual_label};\n\n")
    f.write(f"const float test_data_sample[TEST_DATA_SAMPLE_LENGTH] = {{\n    ")
    f.write(", ".join([f"{val:.8f}f" for val in test_data_sample_raw]))
    f.write("\n};\n")
print(f"Berhasil mengekspor '{source_path}'")

# --- (BARU) VERIFIKASI GROUND TRUTH UNTUK DEBUGGING ---
print("\n--- VERIFIKASI UNTUK DEBUGGING ---")
print("Ini adalah nilai 'processed_sample' yang seharusnya dihasilkan di MCU:")
processed_sample_py = []
for i in range(len(test_data_sample_raw)):
    raw_scaled = int(test_data_sample_raw[i] * SCALE_FACTOR)
    temp = raw_scaled - min_val_scaled[i]
    norm_64 = int(temp) * int(inv_range_val_scaled[i]) # Gunakan perkalian integer murni
    norm_scaled = norm_64 >> 12
    # Tambahkan pembulatan yang benar (add half before shifting)
    quantized_value = (norm_scaled * (256 - 1) + (SCALE_FACTOR // 2)) >> 12

    if quantized_value < 0:
        quantized_value = 0
    elif quantized_value > 255:
        quantized_value = 255
    processed_sample_py.append(int(quantized_value))

print("10 Nilai Pertama 'processed_sample' (Python):")
print(", ".join(map(str, processed_sample_py[:10])))
print("------------------------------------\n")

Memulai ekspor untuk implementasi Fixed-Point...
Data uji yang diekspor adalah untuk kelas: 4
Berhasil mengekspor 'model_params_scaled.h'
Berhasil mengekspor 'test_data.h'
Berhasil mengekspor 'test_data.c'

--- VERIFIKASI UNTUK DEBUGGING ---
Ini adalah nilai 'processed_sample' yang seharusnya dihasilkan di MCU:
10 Nilai Pertama 'processed_sample' (Python):
160, 125, 126, 8, 11, 42, 6, 10, 42, 14
------------------------------------

